In [1]:
!pip install nba_api

In [2]:
%matplotlib inline
import re
import time
import math
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nba_api.stats.endpoints import PlayerGameLogs, TeamGameLogs

In [3]:
def gamedatapull():
    player = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    team = pd.DataFrame()  # Initialize an empty dataframe to store the combined data

    for i in range(2):#24):
        season = f"{2000 + i}-{str(2000 + i + 1)[2:]}" # Right Season

        # Get player game logs data
        player_game_logs = PlayerGameLogs(
            season_nullable=season, # change year(s) if needed
            season_type_nullable='Regular Season') # Regular Season, Playoffs, Pre Season
        df_player_game_logs = player_game_logs.get_data_frames()[0]

        # Get team game logs data
        team_game_logs = TeamGameLogs(
            league_id_nullable='00',  # nba 00, g_league 20, wnba 10
            team_id_nullable='',  # You can specify a specific team_id if needed
            season_nullable=season, # change year(s) if needed
            season_type_nullable='Regular Season') # Regular Season, Playoffs, Pre Season
        df_team_game_logs = team_game_logs.get_data_frames()[0]

        # Concatenate the merged dataframe to the combined dataframe
        player = pd.concat([df_player_game_logs, player], ignore_index=True)
        team =  pd.concat([df_team_game_logs, team], ignore_index=True)

    return player, team

In [4]:
# Call the function to retrieve the combined dataframe
player, T3aM = gamedatapull()

In [5]:
T3aM.to_csv("Team.csv")

In [6]:
# PFTkn
def player_dict(combined_dataframe):
  combined_dataframe['Opposing_Team'] = combined_dataframe['MATCHUP'].str[-3:]
  combined_dataframe['PPP'] = combined_dataframe['PTS'] / (combined_dataframe['FGA'] + 0.44 * combined_dataframe['FTA'] + combined_dataframe['TOV'])
  combined_dataframe['Ast/TO'] = combined_dataframe['AST'].astype(str) + "/" + combined_dataframe['TOV'].astype(str)
  combined_dataframe['BlkFGA'] = combined_dataframe['BLK'].astype(str) + "/" + combined_dataframe['FGA'].astype(str)
  combined_dataframe['FGm'] = combined_dataframe['FGA'] - combined_dataframe['FGM']
  combined_dataframe['2FGA'] =  combined_dataframe['FGA'] - combined_dataframe['FG3A']
  combined_dataframe['2FGM'] = combined_dataframe['FGM'] - combined_dataframe['FG3M']
  combined_dataframe['2FGm'] = combined_dataframe['2FGA'] - combined_dataframe['2FGM']
  combined_dataframe['FG3m'] = combined_dataframe['FG3A'] - combined_dataframe['FG3M']
  combined_dataframe['FTm'] = combined_dataframe['FTA'] - combined_dataframe['FTM']
  combined_dataframe['SST'] = combined_dataframe['STL'].astype(str) + "/" + combined_dataframe['FGA'].astype(str)
  combined_dataframe['Pos'] = (combined_dataframe['FGA'] + 0.44 * combined_dataframe['FTA'] + combined_dataframe['TOV'])
  combined_dataframe['StlPos'] =  (combined_dataframe['STL'] * 100).astype(str) + "/" + combined_dataframe['Pos'].astype(str)
  combined_dataframe['PFCom'] = combined_dataframe['PF']
  combined_dataframe.drop(['Pos', 'AVAILABLE_FLAG'], axis=1, inplace=True)
  return combined_dataframe

In [7]:
player_stats = player_dict(player)
player_stats

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BlkFGA,FGm,2FGA,2FGM,2FGm,FG3m,FTm,SST,StlPos,PFCom
0,2001-02,389,Toni Kukoc,Toni,1610612737,ATL,Atlanta Hawks,0020101176,2002-04-17T00:00:00,ATL @ BOS,...,0/6,4,5,2,3,1,1,1/6,100/8.879999999999999,1
1,2001-02,782,Mitch Richmond,Mitch,1610612747,LAL,Los Angeles Lakers,0020101188,2002-04-17T00:00:00,LAL vs. SAC,...,0/7,4,5,2,3,1,0,0/7,0/7.0,1
2,2001-02,1721,Keon Clark,Keon,1610612761,TOR,Toronto Raptors,0020101178,2002-04-17T00:00:00,TOR vs. CLE,...,0/8,5,8,3,5,0,2,3/8,300/12.76,4
3,2001-02,692,Andrew DeClercq,Andrew,1610612753,ORL,Orlando Magic,0020101180,2002-04-17T00:00:00,ORL vs. MIA,...,1/4,3,4,1,3,0,0,0/4,0/4.0,1
4,2001-02,42,Monty Williams,Monty,1610612753,ORL,Orlando Magic,0020101180,2002-04-17T00:00:00,ORL vs. MIA,...,0/14,5,14,9,5,0,0,1/14,100/15.32,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47876,2000-01,966,Jerome Williams,Jerome,1610612765,DET,Detroit Pistons,0020000005,2000-10-31T00:00:00,DET @ TOR,...,1/2,0,2,2,0,0,0,3/2,300/2.88,5
47877,2000-01,213,Antonio Davis,Antonio,1610612761,TOR,Toronto Raptors,0020000005,2000-10-31T00:00:00,TOR vs. DET,...,2/12,10,12,2,10,0,1,0/12,0/17.759999999999998,3
47878,2000-01,255,Grant Hill,Grant,1610612753,ORL,Orlando Magic,0020000003,2000-10-31T00:00:00,ORL vs. WAS,...,1/6,3,6,3,3,0,2,2/6,200/13.2,3
47879,2000-01,243,Aaron McKie,Aaron,1610612755,PHI,Philadelphia 76ers,0020000001,2000-10-31T00:00:00,PHI @ NYK,...,0/6,2,6,4,2,0,0,0/6,0/7.0,2


In [8]:
def split_game_matchup(game):
    if '@' in game:
        away, home = game.split('@')
    elif 'vs.' in game:
        home, away = game.split('vs.')
    return home.strip(), away.strip()

def convert_to_winner(row):
    if row['WL'] == 'W':
        return row['TEAM_ABBREVIATION']
    else:
        return row['Opposing_Team']

def count_sequences(string):
    # Find all sequences of consecutive 'W's and 'L's
    sequences = re.findall(r'(W+|L+)', string)

    # Count the number of occurrences and sum the lengths for each sequence type
    counts = {'W': 0, 'L': 0}
    for seq in sequences:
        counts[seq[0]] += len(seq)
    tup = str(counts['W']), " - ", str(counts['L'])
    y = ''.join(tup)
    return y

def next_func(df):
  df = df.sort_values('GAME_DATE')
  df['Opposing_Team'] = df['MATCHUP'].str[-3:]

  # Apply the function to create 'Winner' column
  df['Winner'] = df.apply(convert_to_winner, axis=1)

  # Apply the function to split the Game column into Home and Away teams
  df[['H_Team', 'A_Team']] = df['MATCHUP'].apply(lambda x: pd.Series(split_game_matchup(x)))

  df['Record'] = df.groupby(["SEASON_YEAR", "TEAM_NAME"], group_keys=False)['WL'].apply(lambda x: x.cumsum()).apply(count_sequences)
  df = df.sort_values('GAME_ID').reset_index()

  opp = list()
  home = list()
  away = list()
  homep = list()
  awayp = list()
  for i in range(len(df)):
    if i % 2 == 0:
      opp.append(df['Record'][i + 1])
    else:
      opp.append(df['Record'][i - 1])
  df['Opp_Record'] = opp

  for i in range(len(df)):
    if df['H_Team'][i] == df['TEAM_ABBREVIATION'][i]:
      home.append(df['Record'][i])
      away.append(df['Opp_Record'][i])
    else:
      home.append(df['Opp_Record'][i])
      away.append(df['Record'][i])

  df['H_Team_Record'] = home
  df['A_Team_Record'] = away

  return df
def countw(string):
    # Find all sequences of consecutive 'W's and 'L's
    sequences = re.findall(r'(W+|L+)', string)

    # Count the number of occurrences and sum the lengths for each sequence type
    counts = {'W': 0, 'L': 0}
    for seq in sequences:
        counts[seq[0]] += len(seq)
    tup = str(counts['W'])
    y = ''.join(tup)
    return y

def countl(string):
    # Find all sequences of consecutive 'W's and 'L's
    sequences = re.findall(r'(W+|L+)', string)

    # Count the number of occurrences and sum the lengths for each sequence type
    counts = {'W': 0, 'L': 0}
    for seq in sequences:
        counts[seq[0]] += len(seq)
    tup = str(counts['L'])
    y = ''.join(tup)
    return y


def split(df, home, away, target, Home_Col, Away_Col):

  df[Home_Col] = home[target]
  df[Away_Col] = away[target]

  home_list = list()
  away_list = list()

  for i in range(len(df) - 1):
    if pd.isna(df[Home_Col].iloc[i]):
      home_list.append(df[Home_Col][i + 1])

    if pd.isna(df[Away_Col].iloc[i]):
      away_list.append(df[Away_Col][i + 1])

    if not pd.isna(df[Home_Col].iloc[i]):
      home_list.append(df[Home_Col][i])

    if not pd.isna(df[Away_Col].iloc[i]):
      away_list.append(df[Away_Col][i])

  df[Home_Col] = pd.Series(home_list)
  df[Away_Col] = pd.Series(away_list)

  df[Home_Col] = df[Home_Col].fillna(method='ffill', inplace=False)
  df[Away_Col] = df[Away_Col].fillna(method='ffill', inplace=False)

  return df

def record_func(df):
  home = df[(df['TEAM_ABBREVIATION'] == df['H_Team'])]
  df['Home_Wins'] = home[(home['TEAM_ABBREVIATION'] == home['Winner'])].groupby(["SEASON_YEAR", "TEAM_NAME"], group_keys=False)['WL'].apply(lambda x: x.cumsum()).apply(countw)
  df['Home_Loss'] = home[(home['TEAM_ABBREVIATION'] != home['Winner'])].groupby(["SEASON_YEAR", "TEAM_NAME"], group_keys=False)['WL'].apply(lambda x: x.cumsum()).apply(countl)

  away = df[(df['TEAM_ABBREVIATION'] == df['A_Team'])]
  df['Away_Wins'] = away[(away['TEAM_ABBREVIATION'] == away['Winner'])].groupby(["SEASON_YEAR", "TEAM_NAME"], group_keys=False)['WL'].apply(lambda x: x.cumsum()).apply(countw)
  df['Away_Loss'] = away[(away['TEAM_ABBREVIATION'] != away['Winner'])].groupby(["SEASON_YEAR", "TEAM_NAME"], group_keys=False)['WL'].apply(lambda x: x.cumsum()).apply(countl)

  df['Home_Wins'] = df['Home_Wins'].fillna(method='ffill', inplace=False).fillna(0)
  df['Home_Loss'] = df['Home_Loss'].fillna(method='ffill', inplace=False).fillna(0)

  df['Away_Wins'] = df['Away_Wins'].fillna(method='ffill', inplace=False).fillna(0)
  df['Away_Loss'] = df['Away_Loss'].fillna(method='ffill', inplace=False).fillna(0)

  df['Home_Record'] = df['Home_Wins'].astype(str) + ' - ' + df['Home_Loss'].astype(str)
  df['Away_Record'] = df['Away_Wins'].astype(str) + ' - ' + df['Away_Loss'].astype(str)

  # Drop unnecessary columns
  df.drop(['Home_Wins', 'Home_Loss', 'Away_Wins', 'Away_Loss', 'AVAILABLE_FLAG'], axis=1, inplace=True)

  df = split(df, home, away, "PTS", "H_Points", "A_Points")
  df = split(df, home, away, "REB", "H_Total_Rebounds", "A_Total_Rebounds")
  df = split(df, home, away, "OREB", "H_Offensive_Rebounds", "A_Offensive_Rebounds")
  df = split(df, home, away, "DREB", "H_Defensive_Rebounds", "A_Defensive_Rebounds")
  df = split(df, home, away, "AST", "H_Assists", "A_Assists")
  df = split(df, home, away, "TOV", "H_Turnovers", "A_Turnovers")
  df = split(df, home, away, "STL", "H_Steals", "A_Steals")
  df = split(df, home, away, "BLK", "H_Blocks", "A_Blocks")
  df = split(df, home, away, "FTA", "H_All_Free_Throws", "A_All_Free_Throws")
  df = split(df, home, away, "FGA", "H_FG_Attempts", "A_FG_Attempts")
  df = split(df, home, away, "FGM", "H_FG_Made", "A_FG_Made")

  df['FGm'] = df['FGA'] - df['FGM']
  df['FG3m'] = df['FG3A'] - df['FG3M']
  df['2FG_Attempts'] = df['FGA'] - df['FG3A']
  df['2FG_Made'] = df['FGM'] - df['FG3M']
  df['2FG_Missed'] = df['FGm'] - df['FG3m']
  return df
def drop(df):
  home = df[(df['TEAM_ABBREVIATION'] == df['H_Team'])]
  away = df[(df['TEAM_ABBREVIATION'] == df['A_Team'])]
  df = split(df, home, away, "FGm", "H_FG_Missed", "A_FG_Missed")
  df = split(df, home, away, "2FG_Attempts", "H_2FG_Attempts", "A_2FG_Attempts")
  df = split(df, home, away, "2FG_Made", "H_2FG_Made", "A_2FG_Made")
  df = split(df, home, away, "2FG_Missed", "H_2FG_Missed", "A_2FG_Missed")
  df = split(df, home, away, "FG3A", "H_3FG_Attempts", "A_3FG_Attempts")
  df = split(df, home, away, "FG3M", "H_3FG_Made", "A_3FG_Made")
  df = split(df, home, away, "FG3m", "H_3FG_Missed", "A_3FG_Missed")
  df = split(df, home, away, "FG3_PCT", "H_FG3_PCT", "A_FG3_PCT")
  df = split(df, home, away, "FT_PCT", "H_FT_PCT", "A_FT_PCT")
  df = split(df, home, away, "BLKA", "H_BLKA", "A_BLKA")
  df = split(df, home, away, "PLUS_MINUS", "H_PLUS_MINUS", "A_PLUS_MINUS")
  df = split(df, home, away, "FG_PCT", "H_FG_PCT", "A_FG_PCT")
  df = split(df, home, away, "FTM", "H_FTM", "A_FTM")
  df = split(df, home, away, "PF", "H_PF", "A_PF")
  df = split(df, home, away, "PFD", "H_PFD", "A_PFD")

  df.rename(columns={'GAME_DATE': 'Date',  'GAME_ID' : 'Game_ID', 'SEASON_YEAR': 'Season'}, inplace=True)
  df.drop(['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'MIN', 'FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA', 'OREB', 'DREB', 'REB',
           'AST', 'TOV', 'STL', 'BLK', 'PF', 'PFD', 'GP_RANK', 'MIN_RANK', 'FG3_PCT', 'FT_PCT', 'BLKA', 'PFD', 'PLUS_MINUS', 'W_RANK', 'L_RANK',
           'W_PCT_RANK', 'FG_PCT', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
           'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK',
           'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK', 'PTS', 'WL', 'Opposing_Team', 'Record', 'Opp_Record', 'FGm', 'FG3m', '2FG_Attempts',
           '2FG_Made', '2FG_Missed', 'MATCHUP', 'Home_Record', 'Away_Record'], axis=1, inplace=True)

  even_df = df.iloc[::2]
  odd_df = df.iloc[1::2]
  df = pd.merge(odd_df, even_df, on=['Game_ID', 'Date', 'Season', 'H_Team', 'A_Team', 'H_Team_Record', 'A_Team_Record', 
                                     'H_Points', 'A_Points', 'Winner','H_Total_Rebounds', 'A_Total_Rebounds', 
                                     'H_Offensive_Rebounds', 'A_Offensive_Rebounds', 'H_Defensive_Rebounds', 
                                     'A_Defensive_Rebounds', 'H_Assists', 'A_Assists', 'H_Turnovers', 'A_Turnovers', 
                                     'H_Steals', 'A_Steals', 'H_Blocks', 'A_Blocks', 'H_All_Free_Throws', 'A_All_Free_Throws', 'H_FG_Attempts', 'A_FG_Attempts', 'H_FG_Made', 'A_FG_Made',
                                     'H_FG_Missed', 'A_FG_Missed', 'H_2FG_Attempts', 'A_2FG_Attempts', 'H_2FG_Made', 'A_2FG_Made', 'H_2FG_Missed',
                                     'A_2FG_Missed', 'H_3FG_Attempts', 'A_3FG_Attempts', 'H_3FG_Made', 'A_3FG_Made', 'H_3FG_Missed', 'A_3FG_Missed', 'H_FG3_PCT', 'A_FG3_PCT',
                                     'H_FT_PCT', 'A_FT_PCT', 'H_BLKA', 'A_BLKA', 'H_PLUS_MINUS', 'A_PLUS_MINUS', 'H_FG_PCT', 'A_FG_PCT',
                                     "H_FTM", "A_FTM", "H_PF", "A_PF", "H_PFD", "A_PFD"], suffixes=('_o', '_e'), how='inner', validate='one_to_one')

  df.drop(['index_o', 'index_e', 'Winner'], axis=1, inplace=True)
  df['Date'] = pd.to_datetime(df['Date'])
  return df
# HHTR AND AATR
# Double check software

In [9]:
team_stats = drop(record_func(next_func(T3aM)))
team_stats

,Season,Game_ID,Date,H_Team,A_Team,H_Team_Record,A_Team_Record,H_Points,A_Points,H_Total_Rebounds,...,H_PLUS_MINUS,A_PLUS_MINUS,H_FG_PCT,A_FG_PCT,H_FTM,A_FTM,H_PF,A_PF,H_PFD,A_PFD
0,2000-01,0020000002,2000-10-31,NJN,CLE,0 - 1,1 - 0,82.0,86.0,47.0,...,-4.0,4.0,0.365,0.410,17.0,20.0,31.0,27.0,0.0,0.0
1,2000-01,0020000003,2000-10-31,ORL,WAS,1 - 0,0 - 1,97.0,86.0,37.0,...,11.0,-11.0,0.430,0.458,23.0,16.0,24.0,28.0,0.0,0.0
2,2000-01,0020000005,2000-10-31,TOR,DET,0 - 1,1 - 0,95.0,104.0,48.0,...,-9.0,9.0,0.376,0.473,20.0,13.0,21.0,27.0,0.0,0.0
3,2000-01,0020000008,2000-10-31,HOU,MIN,0 - 1,1 - 0,98.0,106.0,38.0,...,-8.0,8.0,0.459,0.489,28.0,14.0,17.0,25.0,0.0,0.0
4,2000-01,0020000010,2000-10-31,UTA,LAC,1 - 0,0 - 1,107.0,94.0,33.0,...,13.0,-13.0,0.506,0.493,16.0,22.0,30.0,32.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,2001-02,0020101182,2002-04-17,IND,PHI,42 - 40,43 - 39,103.0,80.0,44.0,...,23.0,-23.0,0.485,0.446,32.0,14.0,25.0,29.0,0.0,0.0
1177,2001-02,0020101186,2002-04-17,POR,HOU,49 - 33,28 - 54,92.0,79.0,39.0,...,13.0,-13.0,0.475,0.418,11.0,10.0,15.0,17.0,0.0,0.0
1178,2001-02,0020101187,2002-04-17,SEA,MEM,45 - 37,23 - 59,109.0,94.0,36.0,...,15.0,-15.0,0.525,0.500,20.0,15.0,23.0,22.0,0.0,0.0
1179,2001-02,0020101188,2002-04-17,LAL,SAC,58 - 24,61 - 21,109.0,95.0,50.0,...,14.0,-14.0,0.436,0.452,13.0,14.0,23.0,13.0,0.0,0.0


In [15]:
team_stats.columns

Index(['Season', 'Game_ID', 'Date', 'H_Team', 'A_Team', 'H_Team_Record',
       'A_Team_Record', 'H_Points', 'A_Points', 'H_Total_Rebounds',
       'A_Total_Rebounds', 'H_Offensive_Rebounds', 'A_Offensive_Rebounds',
       'H_Defensive_Rebounds', 'A_Defensive_Rebounds', 'H_Assists',
       'A_Assists', 'H_Turnovers', 'A_Turnovers', 'H_Steals', 'A_Steals',
       'H_Blocks', 'A_Blocks', 'H_All_Free_Throws', 'A_All_Free_Throws',
       'H_FG_Attempts', 'A_FG_Attempts', 'H_FG_Made', 'A_FG_Made',
       'H_FG_Missed', 'A_FG_Missed', 'H_2FG_Attempts', 'A_2FG_Attempts',
       'H_2FG_Made', 'A_2FG_Made', 'H_2FG_Missed', 'A_2FG_Missed',
       'H_3FG_Attempts', 'A_3FG_Attempts', 'H_3FG_Made', 'A_3FG_Made',
       'H_3FG_Missed', 'A_3FG_Missed', 'H_FG3_PCT', 'A_FG3_PCT', 'H_FT_PCT',
       'A_FT_PCT', 'H_BLKA', 'A_BLKA', 'H_PLUS_MINUS', 'A_PLUS_MINUS',
       'H_FG_PCT', 'A_FG_PCT', 'H_FTM', 'A_FTM', 'H_PF', 'A_PF', 'H_PFD',
       'A_PFD'],
      dtype='object')

In [15]:
#Home and road team win probabilities implied by Elo ratings and home court adjustment
def win_probs(home_elo, away_elo, home_court_advantage) :
  h = math.pow(10, home_elo/400)
  r = math.pow(10, away_elo/400)
  a = math.pow(10, home_court_advantage/400)

  denom = r + a*h
  home_prob = a*h / denom
  away_prob = r / denom

  return home_prob, away_prob

  #odds the home team will win based on elo ratings and home court advantage

def home_odds_on(home_elo, away_elo, home_court_advantage) :
  h = math.pow(10, home_elo/400)
  r = math.pow(10, away_elo/400)
  a = math.pow(10, home_court_advantage/400)
  return a*h/r

#this function determines the constant used in the elo rating, based on margin of victory and difference in elo ratings
def elo_k(MOV, elo_diff):
  k = 20
  if MOV>0:
      multiplier=(MOV+3)**(0.8)/(7.5+0.006*(elo_diff))
  else:
      multiplier=(-MOV+3)**(0.8)/(7.5+0.006*(-elo_diff))
  return k*multiplier


#updates the home and away teams elo ratings after a game

def update_elo(home_score, away_score, home_elo, away_elo, home_court_advantage) :
  home_prob, away_prob = win_probs(home_elo, away_elo, home_court_advantage)

  if (home_score - away_score > 0) :
    home_win = 1
    away_win = 0
  else :
    home_win = 0
    away_win = 1

  k = elo_k(home_score - away_score, home_elo - away_elo)

  updated_home_elo = home_elo + k * (home_win - home_prob)
  updated_away_elo = away_elo + k * (away_win - away_prob)

  return updated_home_elo, updated_away_elo


#takes into account prev season elo
def get_prev_elo(team, game_date, season, team_stats, elo_df) :
  prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0]

  if team == prev_game['H_Team'] :
    elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['H_Team_Elo_After'].values[0]
  else :
    elo_rating = elo_df[elo_df['Game_ID'] == prev_game['Game_ID']]['A_Team_Elo_After'].values[0]

  if prev_game['Season'] != season :
    return (0.75 * elo_rating) + (0.25 * 1505)
  else :
    return elo_rating

In [16]:
def elo_df_I_teams_elo_df(team_stats):
    team_stats.sort_values(by = 'Date', inplace = True)
    team_stats.reset_index(inplace=True, drop = True)
    elo_df = pd.DataFrame(columns=['Game_ID', 'H_Team', 'A_Team', 'H_Team_Elo_Before', 'A_Team_Elo_Before', 'H_Team_Elo_After', 'A_Team_Elo_After'])
    teams_elo_df = pd.DataFrame(columns=['Game_ID','Team', 'Elo', 'Date', 'Where_Played', 'Season'])

    for index, row in team_stats.iterrows():
      game_id = row['Game_ID']
      game_date = row['Date']
      season = row['Season']
      h_team, a_team = row['H_Team'], row['A_Team']
      h_score, a_score = row['H_Points'], row['A_Points']

      if (h_team not in elo_df['H_Team'].values and h_team not in elo_df['A_Team'].values) :
        h_team_elo_before = 1500
      else :
        h_team_elo_before = get_prev_elo(h_team, game_date, season, team_stats, elo_df)

      if (a_team not in elo_df['H_Team'].values and a_team not in elo_df['A_Team'].values) :
        a_team_elo_before = 1500
      else :
        a_team_elo_before = get_prev_elo(a_team, game_date, season, team_stats, elo_df)

      h_team_elo_after, a_team_elo_after = update_elo(h_score, a_score, h_team_elo_before, a_team_elo_before, 69)

      new_row = {'Game_ID': game_id, 'H_Team': h_team, 'A_Team': a_team, 'H_Team_Elo_Before': h_team_elo_before, 'A_Team_Elo_Before': a_team_elo_before, \
                                                                            'H_Team_Elo_After' : h_team_elo_after, 'A_Team_Elo_After': a_team_elo_after}
      teams_row_one = {'Game_ID': game_id,'Team': h_team, 'Elo': h_team_elo_before, 'Date': game_date, 'Where_Played': 'Home', 'Season': season}
      teams_row_two = {'Game_ID': game_id,'Team': a_team, 'Elo': a_team_elo_before, 'Date': game_date, 'Where_Played': 'Away', 'Season': season}

      elo_df = elo_df.append(new_row, ignore_index = True)
      teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
      teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
    return elo_df, teams_elo_df, team_stats

In [18]:
warnings.filterwarnings("ignore")
elo_df, teams_elo_df, team_stats = elo_df_I_teams_elo_df(team_stats)

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date]

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date]

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0]
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0]
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

...

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date]

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0]
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date][(team_stats['H_Team'] == team) | (team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(1).iloc[0]
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future 

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:31: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  elo_df = elo_df.append(new_row, ignore_index = True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_one, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\2311642443.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  teams_elo_df = teams_elo_df.append(teams_row_two, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1523627199.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game = team_stats[team_stats['Date'] < game_date]

ValueError: not enough values to unpack (expected 3, got 2)

In [21]:
def T_teams_elo_df(teams_elo_df, team_stats):
    dates = list(set([d.strftime("%m-%d-%Y") for d in teams_elo_df["Date"]]))
    dates = sorted(dates, key=lambda x: time.strptime(x, '%m-%d-%Y'))
    teams = team_stats["A_Team"]
    dataset = pd.DataFrame(columns=dates)
    dataset["Team"] = teams.drop_duplicates()
    dataset = dataset.set_index("Team")
    for index, row in teams_elo_df.iterrows():
      date = row["Date"].strftime("%m-%d-%Y")
      team = row["Team"]
      elo = row["Elo"]
      dataset[date][team] = elo

    teams_elo_df['Elo'] = teams_elo_df['Elo'].astype(float)
    return teams_elo_df, team_stats

teams_elo_df, team_stats = T_teams_elo_df(teams_elo_df, team_stats)
elo_df

,Game_ID,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After
0,0020000002,NJN,CLE,1500,1500,1492.435881,1507.564119
1,0020000003,ORL,WAS,1500,1500,1508.852834,1491.147166
2,0020000005,TOR,DET,1500,1500,1488.358072,1511.641928
3,0020000008,HOU,MIN,1500,1500,1489.140894,1510.859106
4,0020000010,UTA,LAC,1500,1500,1509.850900,1490.149100
...,...,...,...,...,...,...,...
1176,0020101182,IND,PHI,1452.094494,1583.163318,1475.847376,1559.410436
1177,0020101187,SEA,MEM,1587.39074,1381.134835,1591.323651,1377.201923
1178,0020101186,POR,HOU,1534.677721,1445.933957,1541.253750,1439.357927
1179,0020101188,LAL,SAC,1621.615161,1669.332408,1634.169201,1656.778367


In [22]:
#given a team and a date, this method will return that teams average stats over the previous n games

def get_avg_stats_last_n_games(team, game_date, season_team_stats, n) :
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)

  h_df = prev_game_df.iloc[:, range(3, 43, 2)]
  h_df.columns = [x[2:] for x in h_df.columns]
  a_df = prev_game_df.iloc[:, range(4, 44, 2)]
  a_df.columns = [x[2:] for x in a_df.columns]

  df = pd.concat([h_df, a_df])
  df = df[df['Team'] == team]
  df.drop(columns = ['Team'], inplace=True)

  return df.mean()

In [23]:
def Recent_performance_df(team_stats):
    recent_performance_df = pd.DataFrame()

    for season in team_stats['Season'].unique() :
      l = ['Date', 'Game_ID', 'Season', 'H_Team', 'A_Team']
      other = list(team_stats.columns[7:60])
      cols = l + other

      season_team_stats = team_stats[team_stats['Season'] == season].sort_values(by = 'Date')[cols].reset_index(drop = True)

      season_recent_performance_df = pd.DataFrame()

      for index, row in season_team_stats.iterrows() :
        game_id = row['Game_ID']
        game_date = row['Date']
        h_team = row['H_Team']
        a_team = row['A_Team']

        h_team_recent_performance = get_avg_stats_last_n_games(h_team, game_date, season_team_stats, 10)
        h_team_recent_performance.index = ['H_Last_10_Avg_' + x for x in h_team_recent_performance.index]

        a_team_recent_performance = get_avg_stats_last_n_games(a_team, game_date, season_team_stats, 10)
        a_team_recent_performance.index = ['A_Last_10_Avg_' + x for x in a_team_recent_performance.index]

        new_row = pd.concat([h_team_recent_performance, a_team_recent_performance], sort=False)
        new_row['Game_ID'] = game_id

        season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
        season_recent_performance_df = season_recent_performance_df[new_row.index]


      recent_performance_df = pd.concat([recent_performance_df, season_recent_performance_df])
    return recent_performance_df, team_stats
recent_performance_df, team_stats = Recent_performance_df(team_stats)

In [24]:
recent_performance_df.dropna()

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

...

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\3689263445.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  prev_game_df = season_team_stats[season_team_stats['Date'] < game_date][(season_team_stats['H_Team'] == team) | (season_team_stats['A_Team'] == team)].sort_values(by = 'Date').tail(n)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1899404969.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  season_recent_performance_df = season_recent_performance_df.append(new_row, ignore_index=True)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_

,H_Last_10_Avg_Points,H_Last_10_Avg_Total_Rebounds,H_Last_10_Avg_Offensive_Rebounds,H_Last_10_Avg_Defensive_Rebounds,H_Last_10_Avg_Assists,H_Last_10_Avg_Turnovers,H_Last_10_Avg_Steals,H_Last_10_Avg_Blocks,H_Last_10_Avg_All_Free_Throws,H_Last_10_Avg_FG_Attempts,...,A_Last_10_Avg_FG_Made,A_Last_10_Avg_FG_Missed,A_Last_10_Avg_2FG_Attempts,A_Last_10_Avg_2FG_Made,A_Last_10_Avg_2FG_Missed,A_Last_10_Avg_3FG_Attempts,A_Last_10_Avg_3FG_Made,A_Last_10_Avg_3FG_Missed,A_Last_10_Avg_FG3_PCT,Game_ID
9,94.0,44.0,12.0,32.0,18.0,24.0,3.0,6.0,34.0,69.0,...,39.0,51.000000,76.0,35.000000,41.000000,14.000000,4.000000,10.0,0.286000,0020000027
20,99.0,33.0,11.0,22.0,20.0,13.0,11.0,5.0,26.0,88.0,...,38.5,48.000000,66.5,31.500000,35.000000,20.000000,7.000000,13.0,0.353500,0020000043
21,106.0,44.0,13.0,31.0,29.0,16.0,7.0,1.0,16.0,88.0,...,34.0,58.000000,79.0,26.000000,53.000000,13.000000,8.000000,5.0,0.615000,0020000041
22,93.0,44.0,14.5,29.5,24.5,17.0,7.0,6.5,26.5,86.0,...,32.0,39.333333,63.0,28.666667,34.333333,8.333333,3.333333,5.0,0.423667,0020000037
23,100.0,49.5,12.0,37.5,23.5,22.0,6.0,6.5,23.0,81.5,...,29.0,37.000000,55.0,25.000000,30.000000,11.000000,4.000000,7.0,0.364000,0020000046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,94.2,41.2,12.5,28.7,19.8,14.2,8.2,5.2,28.6,77.2,...,34.2,42.500000,70.0,32.100000,37.900000,6.700000,2.100000,4.6,0.357000,0020101182
580,93.4,42.5,12.9,29.6,21.9,15.5,9.2,3.7,22.6,80.0,...,37.3,44.200000,65.4,31.300000,34.100000,16.100000,6.000000,10.1,0.362100,0020101186
581,99.0,40.2,12.0,28.2,25.3,13.3,8.6,5.0,19.6,80.7,...,37.7,42.700000,67.5,33.600000,33.900000,12.900000,4.100000,8.8,0.318500,0020101187
582,99.1,44.9,12.7,32.2,19.8,12.2,8.2,6.4,27.8,82.2,...,43.3,44.200000,75.4,39.600000,35.800000,12.100000,3.700000,8.4,0.301100,0020101188


In [25]:
def Final_team_stats(team_stats, elo_df, recent_performance_df):
    final_team_stats = team_stats.iloc[0:, [0,1,2,3,4,7,8]].merge(elo_df.drop(columns=['H_Team', 'A_Team']), on = 'Game_ID') \
                                         .merge(recent_performance_df, on = 'Game_ID')

    final_team_stats = final_team_stats.dropna()
    return final_team_stats

final_team_stats = Final_team_stats(team_stats, elo_df, recent_performance_df)
final_team_stats.head(5)

,Season,Game_ID,Date,H_Team,A_Team,H_Points,A_Points,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,...,A_Last_10_Avg_FG_Attempts,A_Last_10_Avg_FG_Made,A_Last_10_Avg_FG_Missed,A_Last_10_Avg_2FG_Attempts,A_Last_10_Avg_2FG_Made,A_Last_10_Avg_2FG_Missed,A_Last_10_Avg_3FG_Attempts,A_Last_10_Avg_3FG_Made,A_Last_10_Avg_3FG_Missed,A_Last_10_Avg_FG3_PCT
11,2000-01,0020000027,2000-11-02,LAC,VAN,91.0,99.0,1490.1491,1506.216901,1479.829797,...,90.000000,39.0,51.000000,76.0,35.000000,41.000000,14.000000,4.000000,10.0,0.286000
16,2000-01,0020000046,2000-11-04,SEA,POR,90.0,97.0,1503.895837,1483.884949,1493.202536,...,66.000000,29.0,37.000000,55.0,25.000000,30.000000,11.000000,4.000000,7.0,0.364000
17,2000-01,0020000037,2000-11-04,TOR,WAS,103.0,96.0,1484.662418,1502.069701,1491.936319,...,71.333333,32.0,39.333333,63.0,28.666667,34.333333,8.333333,3.333333,5.0,0.423667
18,2000-01,0020000043,2000-11-04,DEN,PHX,99.0,102.0,1489.887263,1512.230365,1483.661646,...,86.500000,38.5,48.000000,66.5,31.500000,35.000000,20.000000,7.000000,13.0,0.353500
19,2000-01,0020000041,2000-11-04,MIN,SAC,99.0,91.0,1510.859106,1510.428217,1518.145285,...,92.000000,34.0,58.000000,79.0,26.000000,53.000000,13.000000,8.000000,5.0,0.615000


In [26]:
def Team_performances_df(final_team_stats, recent_performance_df, teams_elo_df):
    home_cols = final_team_stats.columns[final_team_stats.columns.str.startswith('H_')]

    team_df = final_team_stats.iloc[0:, [0,1,2,3,4,5,6]].drop(columns=['H_Team', 'A_Team'])
    team_df_home = team_df.drop(columns=team_df.columns[team_df.columns.str.startswith('A_')])
    team_df_away = team_df.drop(columns=team_df.columns[team_df.columns.str.startswith('H_')])

    recent_performance_home = recent_performance_df.drop(columns=recent_performance_df.columns \
                                                         [recent_performance_df.columns.str.startswith('A_')])
    recent_performance_away = recent_performance_df.drop(columns=recent_performance_df.columns \
                                                         [recent_performance_df.columns.str.startswith('H_')])

    team_by_team_home = team_df_home.merge(teams_elo_df[teams_elo_df.Where_Played == "Home"], on = 'Game_ID') \
                                         .merge(recent_performance_home, on = 'Game_ID')
    team_by_team_away = team_df_away.merge(teams_elo_df[teams_elo_df.Where_Played == "Away"], on = 'Game_ID') \
                                         .merge(recent_performance_away, on = 'Game_ID')

    team_by_team_home.columns = team_by_team_home.columns.str.replace("H_", "")
    team_by_team_away.columns = team_by_team_away.columns.str.replace("A_", "")
    team_by_team_home.columns = team_by_team_home.columns.str.replace("_y", "")
    team_by_team_home.columns = team_by_team_home.columns.str.replace("_x", "")
    team_by_team_home = team_by_team_home.loc[:,~team_by_team_home.columns.duplicated()]

    team_by_team_away.columns = team_by_team_away.columns.str.replace("_y", "")
    team_by_team_away.columns = team_by_team_away.columns.str.replace("_x", "")
    team_by_team_away = team_by_team_away.loc[:,~team_by_team_away.columns.duplicated()]

    team_performances_df = pd.concat([team_by_team_home, team_by_team_away]).sort_index(axis=0).reset_index().drop(columns=['index'])
    return team_performances_df, final_team_stats 

In [27]:
team_performances_df, final_team_stats = Team_performances_df(final_team_stats, recent_performance_df, teams_elo_df)
team_performances_df

,Season,Game_ID,Date,Points,Team,Elo,Where_Played,Last_10_Avg_Points,Last_10_Avg_Total_Rebounds,Last_10_Avg_Offensive_Rebounds,...,Last_10_Avg_FG_Attempts,Last_10_Avg_FG_Made,Last_10_Avg_FG_Missed,Last_10_Avg_2FG_Attempts,Last_10_Avg_2FG_Made,Last_10_Avg_2FG_Missed,Last_10_Avg_3FG_Attempts,Last_10_Avg_3FG_Made,Last_10_Avg_3FG_Missed,Last_10_Avg_FG3_PCT
0,2000-01,0020000027,2000-11-02,91.0,LAC,1490.149100,Home,94.0,44.0,12.0,...,69.0,34.0,35.0,56.0,30.0,26.0,13.0,4.0,9.0,0.3080
1,2000-01,0020000027,2000-11-02,99.0,VAN,1506.216901,Away,94.0,51.0,14.0,...,90.0,39.0,51.0,76.0,35.0,41.0,14.0,4.0,10.0,0.2860
2,2000-01,0020000046,2000-11-04,97.0,POR,1483.884949,Away,82.0,36.0,10.0,...,66.0,29.0,37.0,55.0,25.0,30.0,11.0,4.0,7.0,0.3640
3,2000-01,0020000046,2000-11-04,90.0,SEA,1503.895837,Home,100.0,49.5,12.0,...,81.5,38.0,43.5,64.0,31.5,32.5,17.5,6.5,11.0,0.3825
4,2000-01,0020000037,2000-11-04,103.0,TOR,1484.662418,Home,93.0,44.0,14.5,...,86.0,34.5,51.5,70.0,30.0,40.0,16.0,4.5,11.5,0.2805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2273,2001-02,0020101177,2002-04-17,112.0,CHH,1523.976358,Home,89.7,42.1,12.6,...,75.4,32.6,42.8,63.5,28.6,34.9,11.9,4.0,7.9,0.3507
2274,2001-02,0020101181,2002-04-17,89.0,MIL,1476.248930,Away,94.3,40.7,9.9,...,79.0,36.4,42.6,60.7,30.8,29.9,18.3,5.6,12.7,0.3050
2275,2001-02,0020101181,2002-04-17,123.0,DET,1536.725322,Home,93.1,37.0,10.1,...,75.5,32.7,42.8,57.4,25.2,32.2,18.1,7.5,10.6,0.4068
2276,2001-02,0020101189,2002-04-17,107.0,GSW,1288.862608,Home,103.9,45.3,15.1,...,89.4,40.2,49.2,72.4,34.2,38.2,17.0,6.0,11.0,0.3361


In [28]:
def Final_Player_Stats1(final_team_stats):
    final_team_stats['Label'] = [1 if x > 0 else 0 for x in final_team_stats['H_Points'] - final_team_stats['A_Points']]
    final_team_stats.drop(columns=['H_Points', 'A_Points'], inplace=True)
    final_team_stats = final_team_stats.dropna()

    combined_copy = final_team_stats
    return final_team_stats, combined_copy

final_team_stats, combined_copy = Final_Player_Stats1(final_team_stats)

final_team_stats

,Season,Game_ID,Date,H_Team,A_Team,H_Team_Elo_Before,A_Team_Elo_Before,H_Team_Elo_After,A_Team_Elo_After,H_Last_10_Avg_Points,...,A_Last_10_Avg_FG_Made,A_Last_10_Avg_FG_Missed,A_Last_10_Avg_2FG_Attempts,A_Last_10_Avg_2FG_Made,A_Last_10_Avg_2FG_Missed,A_Last_10_Avg_3FG_Attempts,A_Last_10_Avg_3FG_Made,A_Last_10_Avg_3FG_Missed,A_Last_10_Avg_FG3_PCT,Label
11,2000-01,0020000027,2000-11-02,LAC,VAN,1490.1491,1506.216901,1479.829797,1516.536204,94.0,...,39.0,51.000000,76.0,35.000000,41.000000,14.000000,4.000000,10.0,0.286000,0
16,2000-01,0020000046,2000-11-04,SEA,POR,1503.895837,1483.884949,1493.202536,1494.578250,100.0,...,29.0,37.000000,55.0,25.000000,30.000000,11.000000,4.000000,7.0,0.364000,0
17,2000-01,0020000037,2000-11-04,TOR,WAS,1484.662418,1502.069701,1491.936319,1494.795799,93.0,...,32.0,39.333333,63.0,28.666667,34.333333,8.333333,3.333333,5.0,0.423667,1
18,2000-01,0020000043,2000-11-04,DEN,PHX,1489.887263,1512.230365,1483.661646,1518.455982,99.0,...,38.5,48.000000,66.5,31.500000,35.000000,20.000000,7.000000,13.0,0.353500,0
19,2000-01,0020000041,2000-11-04,MIN,SAC,1510.859106,1510.428217,1518.145285,1503.142037,106.0,...,34.0,58.000000,79.0,26.000000,53.000000,13.000000,8.000000,5.0,0.615000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1176,2001-02,0020101178,2002-04-17,TOR,CLE,1513.668477,1412.088508,1521.345468,1404.411518,92.8,...,36.0,46.500000,68.5,30.300000,38.200000,14.000000,5.700000,8.3,0.406000,1
1177,2001-02,0020101176,2002-04-17,BOS,ATL,1564.387822,1448.511412,1568.650384,1444.248849,94.4,...,37.0,45.900000,66.1,30.200000,35.900000,16.800000,6.800000,10.0,0.399100,1
1178,2001-02,0020101177,2002-04-17,CHH,CHI,1523.976358,1312.878798,1526.176144,1310.679012,89.7,...,36.1,46.700000,70.8,31.500000,39.300000,12.000000,4.600000,7.4,0.390500,1
1179,2001-02,0020101181,2002-04-17,DET,MIL,1536.725322,1476.24893,1551.436646,1461.537607,93.1,...,36.4,42.600000,60.7,30.800000,29.900000,18.300000,5.600000,12.7,0.305000,1


In [29]:
#final_team_stats.to_csv('Final_Team_Stats.csv')

In [33]:
def Player(player_stats):
    player_stats.rename(columns={'GAME_DATE': 'Date', 'MIN' : 'Min', 'SEASON_YEAR' : 'Season', 'PLAYER_NAME' : 'PlayerName', 'GAME_ID' : 'GameID', 'STL' : 'Stl', 'FG3M' : '3FGM',
                                 'BLK' : 'Blk', 'OREB' : 'OffReb', 'AST' : 'Ast', 'DREB' : 'DefReb', 'TOV' : 'T/O', 'PTS':'Pts'}, inplace=True)
    player_stats = player_stats.sort_values(by = 'Date').reset_index(drop = True)
    player_stats['PPP'] = player_stats['PPP'].apply(lambda x: '0' if x == '-' else x)
    player_stats['PPP'] = player_stats['PPP'].astype(float)
    return player_stats
player_stats = Player(player_stats)

In [34]:
def rolling_average_last_n_games(n) :
  player_stats_recent_performance_df = pd.DataFrame()

  for season in player_stats['Season'].unique() :
    season_player_stats = player_stats[player_stats['Season'] == season]

    for player in season_player_stats['PlayerName'].unique() :
      player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)

      player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)

  player_stats_recent_performance_df.columns = ['Last_' + str(n) + '_Avg_' + x for x in player_stats_recent_performance_df.columns]
  player_stats_recent_performance_df = player_stats[['PlayerName', 'GameID']].merge(player_stats_recent_performance_df.drop(columns='Last_' + str(n) + '_Avg_' + 'GameID'), left_index=True, right_index=True)

  return player_stats_recent_performance_df


In [35]:
warnings.filterwarnings("ignore")
player_recent_performance_df = rolling_average_last_n_games(10)

player_recent_performance_df

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

...

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Ast/TO', 'BlkFGA', 'Date', 'MATCHUP', 'NICKNAME', 'Opposing_Team',
       'PlayerName', 'SST', 'Season', 'StlPos', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'WL'],
      dtype='object')
  player_recent_performance = season_player_stats[season_player_stats['PlayerName'] == player].rolling(n, min_periods=1).mean().shift(1)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_stats_recent_performance_df = player_stats_recent_performance_df.append(player_recent_performance, ignore_index=False)
C:\Users\JaSai\AppData\Local\Temp\ipykernel_48468\1561755695.py:8: Fu

,PlayerName,GameID,Last_10_Avg_PLAYER_ID,Last_10_Avg_TEAM_ID,Last_10_Avg_Min,Last_10_Avg_FGM,Last_10_Avg_FGA,Last_10_Avg_FG_PCT,Last_10_Avg_3FGM,Last_10_Avg_FG3A,...,Last_10_Avg_TD3_RANK,Last_10_Avg_WNBA_FANTASY_PTS_RANK,Last_10_Avg_PPP,Last_10_Avg_FGm,Last_10_Avg_2FGA,Last_10_Avg_2FGM,Last_10_Avg_2FGm,Last_10_Avg_FG3m,Last_10_Avg_FTm,Last_10_Avg_PFCom
0,Mookie Blaylock,0020000011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Devean George,0020000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Jud Buechler,0020000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Ruben Wolkowyski,0020000013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Michael Ruffin,0020000006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47876,Steve Nash,0020101185,959.0,1.610613e+09,30.545167,4.2,10.2,0.3977,1.3,3.7,...,31.0,7992.5,0.766035,6.0,6.5,2.9,3.6,2.4,0.3,1.6
47877,Antonio Daniels,0020101184,1498.0,1.610613e+09,21.129667,2.6,6.0,0.3762,0.4,1.1,...,31.0,15198.7,0.835627,3.4,4.9,2.2,2.7,0.7,0.6,0.8
47878,Jonathan Bender,0020101182,1886.0,1.610613e+09,25.882833,3.4,7.0,0.4565,0.7,1.7,...,31.0,12616.8,0.944880,3.6,5.3,2.7,2.6,1.0,0.6,2.4
47879,Derek Fisher,0020101188,965.0,1.610613e+09,32.106667,4.1,9.9,0.4111,1.9,4.5,...,31.0,9673.9,0.956086,5.8,5.4,2.2,3.2,2.6,0.3,1.9


In [ ]:
avg_season_stats = pd.DataFrame()

for season in player_stats['Season'].unique() :
  season_player_stats = player_stats[player_stats['Season'] == season]
  for player in season_player_stats['PlayerName'].unique() :
    player_avg_season_stats = season_player_stats[season_player_stats['PlayerName'] == player].expanding().mean().shift(1)

    avg_season_stats = avg_season_stats.append(player_avg_season_stats, ignore_index=False)


avg_season_stats.drop(columns= 'GameID', inplace = True)
avg_season_stats.columns = ['Avg_Season_' + x for x in avg_season_stats.columns]
avg_season_stats = player_stats.iloc[:, 0:6].merge(avg_season_stats, left_index = True, right_index = True)

In [ ]:
avg_season_stats

In [ ]:
final_player_stats = avg_season_stats.merge(player_recent_performance_df.drop(columns = ['GameID', 'PlayerName']), left_index=True, right_index=True)
final_player_stats = final_player_stats.dropna()

In [ ]:
final_player_stats.to_csv('Final_Player_Stats.csv')

In [ ]:
labels = []

for i, row in combined_copy.iterrows():
  game_id = row['Game_ID']
  win = row['Label']
  tp_row = team_performances_df.loc[team_performances_df['Game_ID'] == game_id]
  for j, r in tp_row.iterrows():
    #print(r['Where_Played'])
    if r['Where_Played'] == 'Home' and win == 0:
      labels.append(0)
    elif r['Where_Played'] == 'Away' and win == 1:
      labels.append(0)
    else:
      labels.append(1)

team_performances_df['Win'] = labels

team_performances_df['Date'] = pd.to_datetime(team_performances_df['Date'])
team_performances_df.sort_values(by=['Date'], inplace=True, ascending=True)

In [ ]:
team_performances_df

In [ ]:
random_team = random.choice(teams)
data = team_performances_df[team_performances_df['Team'] == random_team]
random_seasons = np.random.choice(list(set(team_performances_df['Season'].values)), size=2, replace=False)
data = data[data['Season'].isin(random_seasons)]


g = sns.jointplot(data = data, x="Elo", y="Last_10_Avg_Points", hue="Season", height=7)
plt.title("{} Plotted ELO vs Average Points for Given Seasons".format(random_team), y=1.3, fontsize = 16)
g.plot_joint(sns.kdeplot, alpha=0.5)

In [ ]:
import matplotlib.patches as mpatches

opponent_avg_points = []
for i, row in data.iterrows():
  game_id = row['Game_ID']
  opp_row = team_performances_df[team_performances_df['Game_ID'] == game_id]
  #print(opp_row.head(2))
  true_row = opp_row[opp_row['Team'] != random_team]
  #print(true_row['Last_10_Avg_Points'].values)
  opponent_avg_points.append(true_row["Last_10_Avg_Points"].values[0])
data['Opp_10_Avg_Points'] = opponent_avg_points

for season in random_seasons:
  fig, ax = plt.subplots(figsize=(7,7))
  new_data = data[data['Season'] == season]
  for a in [new_data['Last_10_Avg_Points'], new_data['Opp_10_Avg_Points']]:
    sns.distplot(a, ax=ax, kde=True)

  colors = ['b', 'r']
  labels = {'r':'Opponent', 'b':random_team }
  handles = []
  for col, patch in zip(colors, ax.axes.patches):
    patch.set_color(col)
    handles.append(mpatches.Patch(color=col, label=labels[col]))

  ax.legend(handles=handles)
  plt.title("{} Average Points vs Opponents in {}".format(random_team, season), fontsize = 16)
  plt.xlabel("Last_10_Avg_Points")

In [ ]:
player_stats['PER_stub'] = player_stats['FGM'] * 85.910 + player_stats['Stl'] * 53.897 + player_stats['3FGM'] * 51.757 + player_stats['FTM'] * 46.845 \
                           + player_stats['Blk'] * 39.190 + player_stats['OffReb'] * 39.190+ player_stats['Ast'] * 34.677+ player_stats['DefReb'] * 14.707 \
                           - player_stats['PFCom'] * 17.17 - player_stats['FTm'] * 20.091 - player_stats['FGm'] * 39.190 - player_stats['T/O'] * 53.897


# raw PER intermediate value
player_stats['PER_stub'] = player_stats['FGM']*85.910+player_stats['Stl']*53.897+player_stats['3FGM']*51.757+player_stats['FTM']*46.845+player_stats['Blk']\
                          *39.190+player_stats['OffReb']*39.19+player_stats['Ast']*34.677+player_stats['DefReb']*14.707-player_stats['PFCom']*17.174-player_stats['FTm']\
                          *20.091-player_stats['FGm']*39.190-player_stats['T/O']*53.897


# calculate PER per player, per game
player_stats['PER'] = player_stats['PER_stub'] * (1 / player_stats['Min'])

In [ ]:
team_per = []
team_per_median = []
for i, row in data.iterrows():
  game_id = row['Game_ID']
  opp_rows = player_stats[player_stats['GameID'] == game_id]
  #print(opp_row.head(2))
  true_rows = opp_rows[opp_rows['TEAM_ABBREVIATION'] == random_team]
  #print(true_row['Last_10_Avg_Points'].values)
  team_per.append(true_rows["PER"].values.mean())
  team_per_median.append(np.median(true_rows["PER"].values))
data['Avg_PER'] = team_per
data["Median_PER"] = team_per_median

g = sns.jointplot(data = data, x="Elo", y="Avg_PER", hue="Season", height=7)
plt.title("{} Plotted ELO vs Average PER for Given Seasons".format(random_team), y=1.3, fontsize = 16)
g.plot_joint(sns.kdeplot, alpha=0.5)

g = sns.jointplot(data = data, x="Elo", y="Median_PER", hue="Season", height=7)
plt.title("{} Plotted ELO vs Median PER for Given Seasons".format(random_team), y=1.3, fontsize = 16)
g.plot_joint(sns.kdeplot, alpha=0.5)

In [ ]:
team_per = []
team_per_median = []
for i, row in data.iterrows():
  game_id = row['Game_ID']
  opp_rows = player_stats[player_stats['GameID'] == game_id]
  #print(opp_row.head(2))
  true_rows = opp_rows[opp_rows['TEAM_ABBREVIATION'] != random_team]
  #print(true_row['Last_10_Avg_Points'].values)
  team_per.append(true_rows["PER"].values.mean())
  team_per_median.append(np.median(true_rows["PER"].values))
data['Opp_Avg_PER'] = team_per
data["Opp_Median_PER"] = team_per_median


for season in random_seasons:
  fig, ax = plt.subplots(figsize=(5,5))
  new_data = data[data['Season'] == season]
  for a in [new_data['Avg_PER'], new_data['Opp_Avg_PER']]:
    sns.distplot(a, ax=ax, kde=True)

  colors = ['b', 'r']
  labels = {'r':'Opponent', 'b':random_team }
  handles = []
  for col, patch in zip(colors, ax.axes.patches):
    patch.set_color(col)
    handles.append(mpatches.Patch(color=col, label=labels[col]))

  ax.legend(handles=handles)
  plt.title("{} Average PER vs Opponents in {}".format(random_team, season), fontsize = 14)
  plt.xlabel("Avg_PER")

In [ ]:
for season in random_seasons:
  fig, ax = plt.subplots(figsize=(5,5))
  new_data = data[data['Season'] == season]
  for a in [new_data['Median_PER'], new_data['Opp_Median_PER']]:
    sns.distplot(a, ax=ax, kde=True)

  colors = ['b', 'r']
  labels = {'r':'Opponent', 'b':random_team }
  handles = []
  for col, patch in zip(colors, ax.axes.patches):
    patch.set_color(col)
    handles.append(mpatches.Patch(color=col, label=labels[col]))

  ax.legend(handles=handles)
  plt.title("{} Median PER vs Opponents in {}".format(random_team, season), fontsize = 14)
  plt.xlabel("Median_PER")

In [ ]:
fts = final_team_stats.copy()
fts['H_Team_Elo_Before'] = fts['H_Team_Elo_Before'].astype(float)
fts['A_Team_Elo_Before'] = fts['A_Team_Elo_Before'].astype(float)
fts['SEASON'] = fts['Season'].str[:4]
fts['Temp'] = fts['Date']
# 0 = Away, 1 = Home
fts.reset_index(inplace=True, drop=True)
for x in range(len(fts)):
    if fts['Label'][x] == 0:
        fts['Label'][x] = fts['A_Team'][x]
    else:
        fts['Label'][x] = fts['H_Team'][x]
#fts['H_Team_Label'] = fts.groupby('H_Team')['Label'].shift(-1)
#fts['A_Team_Label'] = fts.groupby('A_Team')['Label'].shift(-1)
fts.drop(['Season', 'Game_ID', 'H_Team', 'A_Team'],axis=1, inplace=True)
fts['Date'] = fts['Date'] - pd.to_datetime('10-31-' + fts['SEASON'])
fts['SEASON'] = fts['Temp'].dt.year
fts.drop('Temp', axis=1, inplace=True)
fts['Date'] = fts['Date'].dt.days.astype(int)

fts

In [ ]:
features = fts.drop(columns = 'Label')
label = fts['Label']

In [ ]:
features

In [ ]:
from sklearn.model_selection import train_test_split
# Your code goes here

x_train, x_test, y_train, y_test = train_test_split(features, label, test_size = 0.2)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# create a simple, non-parameterized Logistic Regression model
model = LogisticRegression(random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(model.get_params())

In [ ]:
# create complex Logistic Regression with max_iter=131
log_model = LogisticRegression(max_iter=131, verbose=2, random_state=42)
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_log))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Penalty type
penalty = ['l1', 'l2', 'elasticnet', 'none']
# Solver type
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
# Maximum number of iterations
max_iter = [int(x) for x in np.linspace(start = 0, stop = 1000, num = 11)]
# Multi class
multi_class = ['auto', 'ovr', 'multinomial']
# Verbosity
verbose = [0, 1, 2]
# l1 ratio
l1_ratio = [0, 0.25, 0.5, 0.75, 1]
# C
C = [0.5, 1.0, 1.5]


# Create the random grid
random_grid = {
    'penalty': penalty, 'solver': solver, 'max_iter':max_iter,
    'multi_class':multi_class, 'verbose':verbose, 'l1_ratio':l1_ratio,
    'C':C
}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
log_reg = LogisticRegression(random_state=42)
# Random search of parameters, using 2-fold cross validation,
# search across 100 different combinations, and use all available cores
log_random = RandomizedSearchCV(estimator = log_reg, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, n_jobs = -1)
# Fit the random search model
log_random.fit(x_train, y_train)
y_pred_log_random = log_random.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_log_random))

In [ ]:
log_random.best_params_

In [ ]:
basic_model = LogisticRegression(random_state=42)
basic_model.fit(x_train, y_train)
y_pred_basic = basic_model.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_basic))

best_log_random = log_random.best_estimator_
best_log_random.fit(x_train, y_train)
y_pred_best_log_random = best_log_random.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_best_log_random))

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of grid search
# Penalty type
penalty = ['l1', 'l2', 'elasticnet', 'none']
# Solver type
solver = ['lbfgs', 'liblinear']
# Maximum number of iterations
max_iter = [int(x) for x in np.linspace(start = 80, stop = 120, num = 5)]
# Multi class
multi_class = ['auto', 'ovr']
# Verbosity
verbose = [0, 1, 2]
# l1 ratio
l1_ratio = [0, 0.8, 0.9, 1]
# C
C = [0.5, 0.75, 1.0, 1.25, 1.5]

# Create the param grid
param_grid = {'penalty': penalty, 'solver': solver, 'max_iter':max_iter,
    'multi_class':multi_class, 'verbose':verbose, 'l1_ratio':l1_ratio,
    'C':C
}
pprint(param_grid)

# Instantiate the grid search model with 2-fold cross-validation
log_grid_search = GridSearchCV(estimator = LogisticRegression(random_state=42), param_grid = param_grid, cv = 2, n_jobs = -1)

In [ ]:
# Fit the grid search to the data
log_grid_search.fit(x_train, y_train)
best_log_grid = log_grid_search.best_estimator_
best_log_grid.fit(x_train, y_train)
y_pred_best_log = best_log_grid.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_best_log))

In [ ]:
log_grid_search.best_params_

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(x_train, y_train)
y_pred_rf = rf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_rf))

from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 0, stop = 2000, num = 11)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(0, 100, num = 6)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rfc = RandomForestClassifier(random_state=42)
# Random search of parameters, using 2-fold cross validation,
# search across 100 different combinations, and use all available cores
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, n_jobs = -1)
# Fit the random search model
rfc_random.fit(x_train, y_train)
y_pred_rfr_random = rfc_random.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_rfr_random))

In [ ]:
rfc_random.best_params_

In [ ]:
base_model = RandomForestClassifier(random_state=42)
base_model.fit(x_train, y_train)
y_pred_base = base_model.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_base))

best_random = rfc_random.best_estimator_
best_random.fit(x_train, y_train)
y_pred_random = best_random.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred_random))

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of grid search
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1600, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 3)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [3, 5, 8]
# Minimum number of samples required at each leaf node
min_samples_leaf = [3, 4, 5]
# Method of selecting samples for training each tree
bootstrap = [True]
# Create the param grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(param_grid)

# Create a based model
rf = RandomForestClassifier(random_state=42)
# Instantiate the grid search model with 2-fold cross-validation
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 2, n_jobs = -1, verbose = 2)

In [ ]:
h_team_elos = final_team_stats[['Game_ID', 'H_Team', 'H_Team_Elo_Before']].rename(columns = {'H_Team' : 'Team', 'H_Team_Elo_Before': 'Elo_Rating'})
a_team_elos = final_team_stats[['Game_ID', 'A_Team', 'A_Team_Elo_Before']].rename(columns = {'A_Team' : 'Team', 'A_Team_Elo_Before': 'Elo_Rating'})

final_player_stats = final_player_stats.merge(pd.concat([h_team_elos, a_team_elos]), left_on=['GameID', 'OpposingTeam'], right_on=['Game_ID', 'Team']) \
                                                                                                                  .drop(columns = ['Game_ID', 'Team'])

final_player_stats.rename(columns={'Elo_Rating': 'Opp_Team_Elo_Rating'}, inplace = True)

final_player_stats

In [ ]:
final_player_stats = final_player_stats.merge(player_stats[['GameID', 'Pts', 'PlayerName']], left_on=['GameID', 'PlayerName'], right_on=['GameID', 'PlayerName'])
final_player_stats

In [ ]:
game_id_value_counts = final_player_stats['GameID'].value_counts()
valid_game_ids = [x for x in final_player_stats['GameID'] if game_id_value_counts[x] > 16]

final_player_stats = final_player_stats[final_player_stats['GameID'].isin(valid_game_ids)]

In [ ]:
cols = [4,6,8,9,20,21,22,23,24,25,26,27,28,29,30,31,34,36,37,48,49,50,51,52,53,54,55,56,57,58,59, 62]
features = final_player_stats.iloc[:, cols]
label = final_player_stats[['GameID','Pts']]

In [ ]:
features

In [ ]:
import random
games = set(final_player_stats['GameID'].unique())

train_set_game_ids = random.sample(games, 9700)

x_train = features[features['GameID'].isin(train_set_game_ids)].drop(columns = 'GameID')
y_train = label[label['GameID'].isin(train_set_game_ids)].drop(columns = 'GameID')

x_test = features[features['GameID'].isin(train_set_game_ids) == False].drop(columns = 'GameID')
y_test = label[label['GameID'].isin(train_set_game_ids) == False].drop(columns = 'GameID')

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
import numpy as np

model = LinearRegression()
model.fit(x_train, y_train)
predictions = model.predict(x_test)

In [ ]:
print("Score:", model.score(x_test, y_test))
print('MSE:', mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(mean_squared_error(y_test, predictions)))

In [ ]:
#now we will match the predictions up with game and team they came from

y_test['Predicted'] = predictions
predicted_vs_actual_pts_scored = y_test.rename(columns = {'Pts' : 'Actual'})
predicted_player_points = final_player_stats[['GameID', 'PlayerTeam']].merge(predicted_vs_actual_pts_scored, left_index = True, right_index = True)

predicted_team_points = predicted_player_points.groupby(['PlayerTeam', 'GameID']).sum().reset_index().drop(columns = 'Actual')
predicted_team_points.rename(columns = {'PlayerTeam':'Team', 'Predicted' : 'PredictedPts'}, inplace = True)
predicted_team_points

In [ ]:
predicted_winner_df = team_stats.iloc[:,[0,1,3,4,9,10]].merge(predicted_team_points, left_on = ['H_Team', 'Game_ID'], right_on = ['Team', 'GameID']).drop(columns = ['Team', 'GameID'])
predicted_winner_df.rename(columns = {'PredictedPts' : 'H_Predicted_Points'}, inplace = True)

predicted_winner_df = predicted_winner_df.merge(predicted_team_points, left_on = ['A_Team', 'Game_ID'], right_on = ['Team', 'GameID']).drop(columns = ['Team', 'GameID'])
predicted_winner_df.rename(columns = {'PredictedPts' : 'A_Predicted_Points'},inplace=True)

predicted_winner_df

In [ ]:
predicted_winner_df['Actual'] = [1 if x > 0 else 0 for x in predicted_winner_df['H_Points'] - predicted_winner_df['A_Points']]
predicted_winner_df['Predicted'] = [1 if x > 0 else 0 for x in predicted_winner_df['H_Predicted_Points'] - predicted_winner_df['A_Predicted_Points']]

predicted_winner_df

In [ ]:
accuracy = sum(predicted_winner_df['Actual'] == predicted_winner_df['Predicted']) / len(predicted_winner_df)

print('Number of games predicted:', len(predicted_winner_df))
print('Number of correctly predicted game:', sum(predicted_winner_df['Actual'] == predicted_winner_df['Predicted']))
print('Accuracy:', accuracy)